In [ ]:
import numpy as np
import section4_7_m as appendix_M

## 付録K　ファンコンベクター

### K.1 記号及び単位

省略

### K.2 消費電力量

$Q_{T,rad} \leq Q_{min,FC} $ の場合

$$\large
E_{E,rad} = P_{min,FC} \times \frac{Q_{T,H,rad}}{Q_{min,H,FC}} \times 10^{-3}
$$

$Q_{min,FC} < Q_{T,rad} \leq Q_{max,H,FC} $ の場合

$$
E_{E,rad} = \bigg ( P_{min,FC} \times \frac{Q_{max,H,FC}-Q_{T,H,rad}}{Q_{max,H,FC}-Q_{min,H,FC}} + P_{max,FC} \times  \frac{Q_{T,H,rad}-Q_{min,H,FC}}{Q_{max,H,FC}-Q_{min,H,FC}} \bigg) \times 10^{-3}
$$

$Q_{max,H,FC} \leq Q_{T,rad}$ の場合

$$\large
E_{E,rad} = P_{max,FC} \times 10^{-3}
$$

$E_{E,rad}$：1 時間当たりの放熱器の消費電力量(kWh/h)  
$Q_{T,H,rad}$：1 時間当たりの放熱器の処理暖房負荷(MJ/h)  
$Q_{max,H,FC}$：1 時間当たりのファンコンベクターの最大暖房出力(MJ/h)  
$Q_{min,H,FC}$：1 時間当たりのファンコンベクターの最小出力(MJ/h)  
$P_{max,FC}$：ファンコンベクターの最大消費電力(W)  
$P_{min,FC}$：ファンコンベクターの最小消費電力(W)  

In [ ]:
# 放熱器の消費電力量 (1)
def get_E_E_rad(region, mode, A_HCZ, R_type, Theta_SW, Q_T_H_rad):
    # ファンコンベクターの最大能力及び最小能力
    q_max_FC = get_q_max_FC(region, mode, A_HCZ, R_type)
    q_min_FC = get_q_min_FC(q_max_FC)

    # ファンコンベクターの最大暖房出力及び最小暖房出力
    Q_max_H_FC = get_Q_max_H_FC(Theta_SW, q_max_FC)
    Q_min_H_FC = get_Q_min_H_FC(Theta_SW, q_min_FC)

    # ファンコンベクターの最大消費電力及び最小消費電力
    P_max_FC = get_P_max_FC(q_max_FC)
    P_min_FC = get_P_min_FC(q_min_FC)

    # (1a)
    tmp_1a = P_min_FC * (Q_T_H_rad / Q_min_H_FC) * 10 ** (-3)
    tmp_1a[np.logical_not(Q_T_H_rad <= Q_min_H_FC)] = 0

    # (1b)
    tmp_1b = (P_min_FC * (Q_max_H_FC - Q_T_H_rad) / (Q_max_H_FC - Q_min_H_FC) + P_max_FC * (Q_T_H_rad - Q_min_H_FC) / (
                Q_max_H_FC - Q_min_H_FC)) * 10 ** (-3)
    tmp_1b[np.logical_not(np.logical_and(Q_min_H_FC < Q_T_H_rad, Q_T_H_rad < Q_max_H_FC))] = 0

    # (1c)
    tmp_1c = P_max_FC * 10 ** (-3) * np.ones_like(Q_T_H_rad)
    tmp_1c[np.logical_not(Q_max_H_FC <= Q_T_H_rad)] = 0

    E_E_rad = tmp_1a + tmp_1b + tmp_1c
    return E_E_rad


### K.3 温水供給運転率

$$\large
r_{WS,rad} = \frac{Q_{T,H,rad}}{Q_{min,H,FC}}
$$

$r_{WS,rad}$：1 時間平均の放熱器の温水供給運転率  

In [ ]:
# 最大暖房出力
def get_Q_max_H_rad(Theta_SW, q_max_FC):
    return get_Q_max_H_FC(Theta_SW, q_max_FC)

### K.4 最大暖房出力

省略

### K.5 ファンコンベクターの最大暖房出力及び最小暖房出力

$$\large
Q_{max,H,FC} = q_{max,FC} \times \frac{\theta_{SW} - 20}{60 - 20} \times 3600 \times 10^{-6}
$$

$$\large
Q_{min,H,FC} = q_{min,FC} \times \frac{\theta_{SW} - 20}{60 - 20} \times 3600 \times 10^{-6}
$$

$q_{max,FC}$：ファンコンベクターの最大能力(W)  
$q_{min,FC}$：ファンコンベクターの最小能力(W)  
$\theta_{SW}$：往き送水温度(℃)  

In [ ]:
# ファンコンベクターの最大暖房出力 (3a)
def get_Q_max_H_FC(Theta_SW, q_max_FC):
    return q_max_FC * (Theta_SW - 20) / (60 - 20) * 3600 * 10 ** (-6)


# ファンコンベクターの最小暖房出力 (3b)
def get_Q_min_H_FC(Theta_SW, q_min_FC):
    return q_min_FC * (Theta_SW - 20) / (60 - 20) * 3600 * 10 ** (-6)

$$\large
q_{min,FC} = 0.4859 \times q_{max,FC}
$$

In [ ]:
# ファンコンベクターの最大能力
def get_q_max_FC(region, mode, A_HCZ, R_type):
    # 付録Mに定める放熱器の最大能力 q_max_rad に等しいものとする
    return appendix_M.get_q_max_rad(region, mode, A_HCZ, R_type)

### K.6 ファンコンベクターの最大消費電力及び最小消費電力

$$\large
P_{max,FC} = 7.564 \times 10^{-3} \times q_{max,FC}
$$

$$\large
P_{min,FC} = 7.783 \times 10^{-3} \times q_{min,FC}
$$

In [ ]:
# ファンコンベクターの最大能力
def get_q_max_FC(region, mode, A_HCZ, R_type):
    # 付録Mに定める放熱器の最大能力 q_max_rad に等しいものとする
    return appendix_M.get_q_max_rad(region, mode, A_HCZ, R_type)


# ファンコンベクターの最小能力 (4)
def get_q_min_FC(q_max_FC):
    return 0.4859 * q_max_FC